In [ ]:
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    cohen_kappa_score,
    classification_report,
)
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Путь к объединенным CSV-файлам
combined_path = "./combined_csvs/"
combined_files = [f for f in os.listdir(combined_path) if f.endswith(".csv")]

# Сортировка файлов по числу пакетов (например, _250_)
combined_files.sort(key=lambda f: int(re.search(r"(\d+)_packets", f).group(1)))

# Словарь моделей с параметром verbose
models = {
    "Random Forest": RandomForestClassifier(random_state=0, verbose=1, n_jobs=-1),
    "Gradient Boosting": GradientBoostingClassifier(random_state=0, verbose=1),
    "SVM": SVC(probability=True, verbose=1),
    "k-NN": KNeighborsClassifier(n_neighbors=5, n_jobs=-1),
    "Logistic Regression": LogisticRegression(multi_class="multinomial", max_iter=500, verbose=1, n_jobs=-1),
}

# Создание папки для промежуточных файлов
intermediate_path = "./multiclass_models_results/"
os.makedirs(intermediate_path, exist_ok=True)

# Итоговый список для всех результатов
all_results = []

# Проход по каждому CSV-файлу
for combined_file in tqdm(combined_files, desc="Обработка файлов"):
    df = pd.read_csv(os.path.join(combined_path, combined_file))
    X = df.drop(columns=["Label"])
    y = df["Label"]

    # Извлечение размера пакетов из имени файла
    packet_size = int(re.search(r"(\d+)_packets", combined_file).group(1))

    # Разделение данных и нормализация в отдельном блоке try
    try:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=0, stratify=y
        )
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    except ValueError as e:
        print(f"  ! Ошибка при подготовке данных для файла '{combined_file}': {e}")
        continue

    # Проход по каждой модели
    for model_name, model in models.items():
        results = {
            "Model": model_name,
            "File": combined_file,
            "Packet Size": packet_size,
        }

        print(f"  - Обучение модели {model_name}")
        try:
            # Обучение модели с выводом прогресса
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Расчёт метрик
            results["Accuracy"] = accuracy_score(y_test, y_pred)
            results["Macro F1 Score"] = f1_score(y_test, y_pred, average="macro")
            results["Micro F1 Score"] = f1_score(y_test, y_pred, average="micro")
            results["Weighted F1 Score"] = f1_score(y_test, y_pred, average="weighted")
            results["Macro Precision"] = precision_score(
                y_test, y_pred, average="macro"
            )
            results["Micro Precision"] = precision_score(
                y_test, y_pred, average="micro"
            )
            results["Cohen's Kappa"] = cohen_kappa_score(y_test, y_pred)

            # Отчёт по каждому классу
            class_report = classification_report(y_test, y_pred, output_dict=True)
            for label, metrics in class_report.items():
                if isinstance(metrics, dict):
                    results[f"Precision_{label}"] = metrics["precision"]
                    results[f"Recall_{label}"] = metrics["recall"]
                    results[f"F1_{label}"] = metrics["f1-score"]

        except ValueError as e:
            # Пропуск метрик при ошибке
            results.update(
                {
                    "Accuracy": "-",
                    "Macro F1 Score": "-",
                    "Micro F1 Score": "-",
                    "Weighted F1 Score": "-",
                    "Macro Precision": "-",
                    "Micro Precision": "-",
                    "Cohen's Kappa": "-",
                }
            )
            print(
                f"  ! Ошибка обучения модели '{model_name}' для файла '{combined_file}': {e}"
            )

        # Добавление результата
        all_results.append(results)

        # Промежуточный CSV для модели
        model_results_df = pd.DataFrame([results])
        model_results_file = os.path.join(
            intermediate_path, f"{model_name}_multiclass_results.csv"
        )

        # Если файл существует, добавляем данные
        if os.path.exists(model_results_file):
            model_results_df.to_csv(
                model_results_file, mode="a", index=False, header=False
            )
        else:
            model_results_df.to_csv(model_results_file, index=False)

# Итоговый файл с результатами всех моделей
final_results_df = pd.DataFrame(all_results)
final_results_df.to_csv("final_multiclass_models_results.csv", index=False)
print("Итоговый файл сохранён в 'final_multiclass_models_results.csv'")